# Delineation of field boundary

If you are using python for spatial processing, it is sometimes useful to plot your data. In these cases, it may also be helpful to render a simple map to help locate the data. This notebook will show you how plot a shapefile using pyplot for this purpose.

## Dependencies

If you don't have `pyshp` or `pyproj` in your jupyter environement, install them now:

In [1]:
!pip install pyshp pyproj

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


## Loading the Shapefile

### Pre-requisite
Download data from [here](https://drive.google.com/file/d/1ezdwnKeJ1rbxmiJznhOCaImNQ02KPraZ/view?usp=sharing) and save in `FR_ROI_fields_2019/`
We will use the shapefile from the link above.


In [5]:
import shapefile
import matplotlib.pyplot as plt

fp = "FR_ROI_fields_2019/FR_ROI_parcels.shp"
sf = shapefile.Reader(fp)
print(sf.fields)


[('DeletionFlag', 'C', 1, 0), ['fid_', 'N', 10, 0], ['ID_PARCEL', 'C', 10, 0], ['SURF_PARC', 'F', 19, 11], ['CODE_CULTU', 'C', 3, 0], ['CODE_GROUP', 'C', 2, 0], ['CULTURE_D1', 'C', 3, 0], ['CULTURE_D2', 'C', 3, 0]]


In [6]:
import geopandas as gpd
gpd.read_file(fp)

,fid_,ID_PARCEL,SURF_PARC,CODE_CULTU,CODE_GROUP,CULTURE_D1,CULTURE_D2,geometry
0,0,8098361,2.38,ORP,3,DVS,DMD,"POLYGON ((710695.531 6809279.064, 710701.676 6..."
1,0,8098358,1.37,ORH,3,DVS,DMD,"POLYGON ((712269.586 6810265.643, 712306.099 6..."
2,0,8066789,0.43,J6S,11,None,None,"POLYGON ((721420.806 6816453.165, 721385.289 6..."
3,0,13540065,10.25,BTH,1,DPH,DMD,"POLYGON ((701787.638 6792938.984, 701813.697 6..."
4,0,8047083,1.39,J6S,11,None,None,"POLYGON ((720633.951 6809664.312, 720870.047 6..."
...,...,...,...,...,...,...,...,...
718263,0,16261356,9.66,SPL,17,None,None,"POLYGON ((779239.337 6383326.171, 779235.898 6..."
718264,0,16261833,8.00,SPL,17,None,None,"POLYGON ((779236.691 6383320.849, 779215.524 6..."
718265,0,16262868,20.67,BOP,17,None,None,"POLYGON ((778989.692 6383116.464, 779157.306 6..."
718266,0,16263366,2.07,SPL,17,None,None,"POLYGON ((779197.473 6383212.534, 779213.613 6..."


## Reprojection

The field boundary shapefile is in the RGF93 / Lambert-93 projetion (`EPSG:2154`). We will reproject to Web Mercator (`EPSG:3857`) for visualization (may not need). To do this, we construct a `Transformer` using `pyproj`.

At the same time, set up the rendering bounds. For simplicity, define the bounds in lat/lon and reproject to meters. Note that `pyplot` expects bounds in `minx,maxx,miny,maxy` order, while `pyproj` transform works on in `x,y` pairs.

In [7]:
import pyproj

transformer = pyproj.Transformer.from_crs('EPSG:2154', 'EPSG:3857', always_xy=True)

BOUNDS = [3,44,49,46.5] # Need to verify. Got this data from FR_ROI_parcels.prj
BOUNDS[0],BOUNDS[2] = transformer.transform(BOUNDS[0],BOUNDS[2])
BOUNDS[1],BOUNDS[3] = transformer.transform(BOUNDS[1],BOUNDS[3])

## Plotting the Data

To display the shapefile, iterate through the shapes in the shapefile. Collect the points for each shape, and transform them to `EPSG:3857` using the transformer constructed above. Plot each shape with pyplot using `fill` for the fill and `plot` for the outline.

Finaly, set the bounds on the plot.

In [8]:
%matplotlib notebook
import matplotlib.pyplot as plt

for shape in sf.shapeRecords():
    for i in range(len(shape.shape.parts)):
        # fillcolor=mapcolor_map[shape.record[mapcolor_idx]]
        
        i_start = shape.shape.parts[i]
        if i==len(shape.shape.parts)-1:
            i_end = len(shape.shape.points)
        else:
            i_end = shape.shape.parts[i+1]
        points = list(transformer.itransform(shape.shape.points[i_start:i_end]))

        x = [i[0] for i in points]
        y = [i[1] for i in points]
        #Poly Fill
        # plt.fill(x,y, facecolor=fillcolor, alpha=0.8)
        plt.fill(x,y, alpha=0.8)
        #Poly line
        plt.plot(x,y, color='#000000', alpha=1, linewidth=1)
        
ax = plt.axis(BOUNDS)

KeyboardInterrupt: 